In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()



True

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from docx import Document as DocxDocument
from PyPDF2 import PdfReader
import os

def load_and_process_document(file_path):
    # Determine file extension
    _, file_extension = os.path.splitext(file_path)
    
    if file_extension.lower() == '.txt':
        loader = TextLoader(file_path)
        documents = loader.load()
    elif file_extension.lower() == '.pdf':
        documents = load_pdf(file_path)
    elif file_extension.lower() == '.docx':
        documents = load_docx(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    docs = text_splitter.split_documents(documents)

    return docs 

def load_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + '\n'
    return [{'text': text}]

def load_docx(file_path):
    doc = DocxDocument(file_path)
    text = ''
    for para in doc.paragraphs:
        text += para.text + '\n'
    return [{'text': text}]


In [3]:
load_and_process_document(r"C:\Users\thibe\OneDrive\Documents\Hackathon\plumber.txt")

([Document(page_content="# Plumber Services Document\n\n## Introduction\nWelcome to Local Plumber Services, your reliable partner for all your plumbing needs. With years of experience, our team of licensed and insured plumbers is dedicated to providing top-notch service to both residential and commercial clients. From minor repairs to major installations, we handle it all with professionalism and efficiency. Below, you'll find detailed information about our services, pricing, frequently asked questions, and more.\n\n## Services Offered", metadata={'source': 'C:\\Users\\thibe\\OneDrive\\Documents\\Hackathon\\plumber.txt'}),
  Document(page_content='### Residential Plumbing Services\n- **Leak Detection and Repair:** Identifying and fixing leaks in pipes, faucets, and fixtures.\n- **Drain Cleaning:** Clearing clogs and ensuring smooth drainage in sinks, tubs, and toilets.\n- **Pipe Installation and Repair:** Installing new pipes or repairing existing ones.\n- **Water Heater Services:** In

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader(r"C:\Users\thibe\OneDrive\Documents\Hackathon\plumber.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [3]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key= os.getenv('PINECONE_API_KEY'))

c:\Users\thibe\Anaconda3\envs\ttt\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
from langchain_pinecone import PineconeVectorStore  
text_field = "text"  
vectorstore = PineconeVectorStore(  
    index_name, embeddings  
)  


In [9]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)


In [11]:
query = "what are the closing times ?"  
vectorstore.similarity_search(query, k = 2)


[Document(page_content="## Opening Times\n- **Monday to Friday:** 8:00 AM - 6:00 PM\n- **Saturday:** 9:00 AM - 4:00 PM\n- **Sunday:** Closed\n- **Emergency Services:** Available 24/7\n\n## Frequently Asked Questions (FAQ)\n\n### 1. Do you offer free estimates?\nYes, we offer free estimates for all our services. Simply contact us to schedule an appointment, and one of our experienced plumbers will assess your needs and provide a detailed quote.\n\n### 2. Are your plumbers licensed and insured?\nAbsolutely. All our plumbers are fully licensed, insured, and trained to handle a wide range of plumbing issues. Your safety and satisfaction are our top priorities.\n\n### 3. What areas do you serve?\nWe serve a wide range of areas including [List of Cities/Regions]. If you're unsure whether we service your area, please give us a call, and we'll be happy to assist.\n\n### 4. What types of payment do you accept?\nWe accept various forms of payment including cash, checks, major credit cards, and o

In [16]:
from langchain_groq import ChatGroq
from langchain.schema import Document 
from langchain.chains import RetrievalQA  
from langchain_openai import OpenAIEmbeddings

In [15]:
from api.v1.config.mongodb_config import MongoDBConfig


In [19]:
import os

In [30]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


In [36]:
import os
import numpy as np
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
 
# Define your query and top_k
query = "What are the closing times?"
top_k = 2

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Check the dimensionality of the embeddings
sample_embedding = embeddings.embed_query(query)  # or embeddings.embed_documents([query])
embedding_dim = np.array(sample_embedding).shape[-1]
print(f"Embedding dimensionality: {embedding_dim}")

# Ensure your Pinecone index is created with the correct dimensionality
mongodb_config = MongoDBConfig()

# Search Index name
index_name = mongodb_config.vector_search_index_name

vectorstore = PineconeVectorStore(index_name="langchain-index",
                                  embedding=embeddings)

# Initialize the completion LLM
llm = ChatGroq(
    temperature=0,
    model_name='mixtral-8x7b-32768',
    groq_api_key=os.getenv('GROQ_API_KEY')
)

# Define a custom prompt template
custom_prompt_template = """
You are an AI assistant. Answer the following question based on the provided context and your knowledge.

Context: {context}

Question: {question}

Answer:
"""

prompt = PromptTemplate(template=custom_prompt_template, input_variables=["context", "question"])

# Set up memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create the conversational retrieval chain with the custom prompt
chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    prompt_template=prompt
)

# Run the chain with the given query
response = chain.run({'question': query})
print(response)


c:\Users\thibe\Anaconda3\envs\ttt\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Embedding dimensionality: 1536


ValidationError: 1 validation error for ConversationalRetrievalChain
prompt_template
  extra fields not permitted (type=value_error.extra)

In [32]:
query = "what are the closing times ?"
 
top_k = 2
embeddings: OpenAIEmbeddings = OpenAIEmbeddings()
mongodb_config: MongoDBConfig = MongoDBConfig()

# Search Index name
index_name: str = mongodb_config.vector_search_index_name

    
vectorstore = PineconeVectorStore(index_name="langchain-index", 
                                    embedding=embeddings)



# completion llm  
llm = ChatGroq(  
    temperature = 0, 
    model_name = 'mixtral-8x7b-32768',
    groq_api_key = os.getenv('GROQ_API_KEY')
)  

memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)
chain = ConversationalRetrievalChain.from_llm(llm, retriever= vectorstore.as_retriever() , memory= memory)

 
chain.run({'question': query}) 

'Based on the information provided, the closing times for this business are as follows:\n\n- Monday to Friday: 6:00 PM\n- Saturday: 4:00 PM\n- Sunday: The business is closed on Sundays.\n\nFor emergency services, they are available 24/7. You can call their emergency hotline if you need assistance outside of their regular business hours.'

In [34]:
chain.run({'question': "how are you?"}) 

'Hello! Based on the provided context, Local Plumber Services offers a variety of plumbing services. These include:\n\n1. Free estimates for all services.\n2. Handling of plumbing emergencies available 24/7.\n3. Services for bathroom and kitchen remodels.\n4. Annual plumbing system inspections.\n5. Warranty on parts and labor for all services.\n6. Assistance with clogged drains.\n\nIf you have any specific questions or need more information, feel free to ask!'

In [26]:
vectorstore.similarity_search(query, 
                                k = 2)

[Document(page_content="## Opening Times\n- **Monday to Friday:** 8:00 AM - 6:00 PM\n- **Saturday:** 9:00 AM - 4:00 PM\n- **Sunday:** Closed\n- **Emergency Services:** Available 24/7\n\n## Frequently Asked Questions (FAQ)\n\n### 1. Do you offer free estimates?\nYes, we offer free estimates for all our services. Simply contact us to schedule an appointment, and one of our experienced plumbers will assess your needs and provide a detailed quote.\n\n### 2. Are your plumbers licensed and insured?\nAbsolutely. All our plumbers are fully licensed, insured, and trained to handle a wide range of plumbing issues. Your safety and satisfaction are our top priorities.\n\n### 3. What areas do you serve?\nWe serve a wide range of areas including [List of Cities/Regions]. If you're unsure whether we service your area, please give us a call, and we'll be happy to assist.\n\n### 4. What types of payment do you accept?\nWe accept various forms of payment including cash, checks, major credit cards, and o

In [21]:
answer

'Based on the information provided, the closing times for this business are as follows:\n\n- Monday to Friday: 6:00 PM\n- Saturday: 4:00 PM\n- Sunday: The business is closed on Sundays.\n\nFor emergency services, they are available 24/7. You can call their emergency hotline if you need assistance outside of their regular business hours.'

In [3]:
index_name = "langchain-index"

In [4]:
import time

index_name = "langchain-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [5]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [8]:
query = "What is the price for emergency repairs?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

## Pricing

### Residential Pricing
- **Service Call Fee:** R75 (waived with repair)
- **Hourly Rate:** R100 per hour
- **Drain Cleaning:** Starting at R150
- **Leak Detection and Repair:** Starting at R125
- **Water Heater Installation:** Starting at R500
- **Fixture Installation:** Starting at R200 per fixture
- **Sewer Line Repair:** Starting at R1,000

### Commercial Pricing
- **Service Call Fee:** R100 (waived with repair)
- **Hourly Rate:** R150 per hour
- **Routine Maintenance Packages:**
  - **Monthly:** R250
  - **Quarterly:** R600
  - **Annual:** R2,200
- **Backflow Prevention:** Starting at R300
- **Grease Trap Cleaning:** Starting at R200

### Emergency Services
- **After-Hours Service Call Fee:** R150
- **Emergency Hourly Rate:** R200 per hour

*Please note that all prices are estimates and may vary based on the specific requirements of each job. We provide detailed quotes before starting any work.*
